# spark ことはじめ

## sparkについて
spark はサイズの大きいデータを扱うための分散処理フレームワーク。  
HadoopのようなものでHadoopはJavaであるのに対し、sparkはscalaで記述されている。
scala, java python, RなどでAPIが利用できる。

sparkはHadoop内でのMapReduceの部分にあたる。  
Hadoopと違い、メモリ上で実行するのでHadoopよりも早い。（Prestoのようなもの？）


## トピックス
調べたこと

### RDD(Resilient Distributed Datasets)
RDDとは、耐障害性分散データセットで__繰り返し使用するデータをメモリ上に保持する__。  

* 遅延評価（特定のメソッドが呼ばれるまで、データは処理されない）
* 読み取り専用
* イミュータブルである
* 再利用のためにメモリ上にキャッシュされる

In [21]:
import sys

import numpy as np
import pandas as pd

from sklearn.datasets import load_boston

In [134]:
from pyspark.sql import SparkSession
# セッションを開く
try:
    spark.stop()
except NameError:
    pass

spark = SparkSession.builder.appName("pyspark-notebook").master(master="local[*]").getOrCreate()
spark

In [135]:
"""
バージョン確認
"""
print (f"Python version : {sys.version}")
print (f"Spark version : {spark.version}")



Python version : 3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]
Spark version : 3.1.1


In [136]:
#bostonデータを取得し、sparkのdataframeへ変換
boston_data = load_boston()

boston_df = pd.DataFrame(boston_data["data"], columns=boston_data["feature_names"])
boston_df["target"] = boston_data["target"]
feature_cols = boston_data["feature_names"]
boston_df = spark.createDataFrame(boston_df)
boston_df.show(10)

print(type(boston_df))

+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|target|
+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+------+
|0.00632|18.0| 2.31| 0.0|0.538|6.575| 65.2|  4.09|1.0|296.0|   15.3| 396.9| 4.98|  24.0|
|0.02731| 0.0| 7.07| 0.0|0.469|6.421| 78.9|4.9671|2.0|242.0|   17.8| 396.9| 9.14|  21.6|
|0.02729| 0.0| 7.07| 0.0|0.469|7.185| 61.1|4.9671|2.0|242.0|   17.8|392.83| 4.03|  34.7|
|0.03237| 0.0| 2.18| 0.0|0.458|6.998| 45.8|6.0622|3.0|222.0|   18.7|394.63| 2.94|  33.4|
|0.06905| 0.0| 2.18| 0.0|0.458|7.147| 54.2|6.0622|3.0|222.0|   18.7| 396.9| 5.33|  36.2|
|0.02985| 0.0| 2.18| 0.0|0.458| 6.43| 58.7|6.0622|3.0|222.0|   18.7|394.12| 5.21|  28.7|
|0.08829|12.5| 7.87| 0.0|0.524|6.012| 66.6|5.5605|5.0|311.0|   15.2| 395.6|12.43|  22.9|
|0.14455|12.5| 7.87| 0.0|0.524|6.172| 96.1|5.9505|5.0|311.0|   15.2| 396.9|19.15|  27.1|
|0.21124|12.5| 7.87| 

#### TransformerとAction
sparkにおいてrddを直接扱うのは理想ではない。(sparkが最適化できる余地を無くすため)  
`spark.sql.functions`を利用するのが理想。

Actionは実行されるたびにRDDが計算されるので、適宜中間データを`cache` or `永続化`をした方がいい場合がある。

In [137]:
import pyspark.sql.functions as F

In [138]:
### カラム同士の和
#rddを使った演算
%time boston_df.rdd.map(lambda x : [x.target + x.LSTAT]).toDF(["hoge"]).select(["hoge"]).show(10)

#sql functionsの演算
%time boston_df.withColumn("hoge", F.col("target") + F.col("LSTAT")).select(["hoge"]).show(10)

#元のテーブルを変更するわけではない
boston_df.show(10)

+------------------+
|              hoge|
+------------------+
|             28.98|
|30.740000000000002|
|38.730000000000004|
|36.339999999999996|
|             41.53|
|             33.91|
|             35.33|
|             46.25|
|             46.43|
|              36.0|
+------------------+
only showing top 10 rows

CPU times: user 7.36 ms, sys: 2.39 ms, total: 9.75 ms
Wall time: 136 ms
+------------------+
|              hoge|
+------------------+
|             28.98|
|30.740000000000002|
|38.730000000000004|
|36.339999999999996|
|             41.53|
|             33.91|
|             35.33|
|             46.25|
|             46.43|
|              36.0|
+------------------+
only showing top 10 rows

CPU times: user 2.29 ms, sys: 635 µs, total: 2.92 ms
Wall time: 37.4 ms
+-------+----+-----+----+-----+-----+-----+------+---+-----+-------+------+-----+------+
|   CRIM|  ZN|INDUS|CHAS|  NOX|   RM|  AGE|   DIS|RAD|  TAX|PTRATIO|     B|LSTAT|target|
+-------+----+-----+----+-----+-----+-

In [139]:
import string

from pyspark.sql import Row

digits = string.digits
lowercase = string.ascii_lowercase

np.random.choice(list(digits), 3)

rows = [
    Row(
        col1="".join(np.random.choice(list(digits), 3))
        , col2="".join(np.random.choice(list(lowercase), 3))
        , col3=[
            "".join(np.random.choice(list(digits+lowercase), 3))
            , "".join(np.random.choice(list(digits+lowercase), 3))
        ]
    ) for _ in range(10)
]

rdd = spark.sparkContext.parallelize(rows)

array_df = spark.createDataFrame(rdd)
array_df.show()

+----+----+----------+
|col1|col2|      col3|
+----+----+----------+
| 616| mjl|[hab, ayc]|
| 757| fcq|[jup, zn1]|
| 483| hjf|[o9h, 11g]|
| 855| rrm|[vr6, g7q]|
| 060| ovq|[pex, 72f]|
| 135| fly|[sla, 1jh]|
| 128| ssc|[h92, 7v6]|
| 438| dss|[yff, gnk]|
| 857| nzp|[993, 32n]|
| 774| lhx|[wbk, h7m]|
+----+----+----------+



In [140]:
#rddを使った演算
%time array_df.rdd.map(lambda x: [x.col3[0], x.col3[1]]).toDF(["arr_0", "arr_1"]).show()

#sql functionsの演算
%time array_df.withColumn("arr_0", array_df["col3"].getItem(0)).withColumn("arr_1", array_df["col3"].getItem(1)).show()

+-----+-----+
|arr_0|arr_1|
+-----+-----+
|  hab|  ayc|
|  jup|  zn1|
|  o9h|  11g|
|  vr6|  g7q|
|  pex|  72f|
|  sla|  1jh|
|  h92|  7v6|
|  yff|  gnk|
|  993|  32n|
|  wbk|  h7m|
+-----+-----+

CPU times: user 6.93 ms, sys: 2.4 ms, total: 9.32 ms
Wall time: 136 ms
+----+----+----------+-----+-----+
|col1|col2|      col3|arr_0|arr_1|
+----+----+----------+-----+-----+
| 616| mjl|[hab, ayc]|  hab|  ayc|
| 757| fcq|[jup, zn1]|  jup|  zn1|
| 483| hjf|[o9h, 11g]|  o9h|  11g|
| 855| rrm|[vr6, g7q]|  vr6|  g7q|
| 060| ovq|[pex, 72f]|  pex|  72f|
| 135| fly|[sla, 1jh]|  sla|  1jh|
| 128| ssc|[h92, 7v6]|  h92|  7v6|
| 438| dss|[yff, gnk]|  yff|  gnk|
| 857| nzp|[993, 32n]|  993|  32n|
| 774| lhx|[wbk, h7m]|  wbk|  h7m|
+----+----+----------+-----+-----+

CPU times: user 1.64 ms, sys: 419 µs, total: 2.06 ms
Wall time: 59.2 ms


In [ ]:
spark.s